In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from utils import *

ModuleNotFoundError: No module named 'utils'

# Weekly activities
1. Apply custom sharpening kernel of aperture size 3 and 5 as shown below on 'native-bee.png':  
$ 3 \times 3$ kernel:  
$ \begin{bmatrix}
0 & -1 & 0 \\
-1 & 5 & -1 \\
0 & -1 & 0\\
\end{bmatrix}$  
$ 5 \times 5$ kernel:  
$ \begin{bmatrix}
-1 & -1 & -1 & -1 & -1 \\
-1 & -1 & -1 & -1 & -1 \\
-1 & -1 & 25 & -1 & -1 \\
-1 & -1 & -1 & -1 & -1 \\
-1 & -1 & -1 & -1 & -1 \\
\end{bmatrix}$  
What can you infer from the outputs?
2. Apply different image smoothing techniques (e.g. average filter, Gaussian kernel and median filter) on 'noise_lena.jpg' and display the resulting images after the convolution. Comment on the outcomes and deduce the type of noise present on the image.
3. Write a program to *segment the boat and the people on it from the background*. Follow the instruction below:
    - Use 'boat.jpg' as input.
    - Apply Otsu thresholding.
    - Draw bounding box to identify the region where the boat and people are located.

In [ ]:
# Question 1 
img = cv.imread("images/native-bee.png")

# Define the kernels
kernel_3x3 = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])

kernel_5x5 = np.array([[-1, -1, -1, -1, -1],
                       [-1, -1, -1, -1, -1],
                       [-1, -1, 25, -1, -1],
                       [-1, -1, -1, -1, -1],
                       [-1, -1, -1, -1, -1]])

# Apply the kernels
dst_3x3 = cv.filter2D(img, -1, kernel_3x3)
dst_5x5 = cv.filter2D(img, -1, kernel_5x5)

# Display the images
display_images([img, dst_3x3, dst_5x5], ("Original", "3x3 Sharpened", "5x5 Sharpened"))



3x3 Kernel:
This kernel is performing as intended. It's sharpening the image by enhancing edges and fine details without causing excessive artifacts. The 3x3 kernel is generally considered a good balance for sharpening, as it improves image clarity without being too aggressive. 
<br>

5x5 Kernel:
I've observed that this kernel is "overly sharpening" the image. This is due to its more aggressive nature:
The central value (25) is much higher compared to the surrounding values (-1), creating a very strong contrast.
The larger size (5x5) affects a wider area around each pixel, leading to more pronounced effects.

In [ ]:
# Question 2
img = cv.imread("images/noise_lena.jpg")

# Apply average filter
avg_filter = cv.blur(img, (5,5))

# Apply Gaussian filter
gaussian_filter = cv.GaussianBlur(img, (5,5), 0)

# Apply median filter
median_filter = cv.medianBlur(img, 5)

# Display the images
display_images([img, avg_filter, gaussian_filter, median_filter], 
               ("Original", "Average Filter", "Gaussian Filter", "Median Filter"))

Average Filter:
The fact that the image is a bit blurry but still contains noise suggests that the average filter is struggling to completely remove the noise. This is typical when dealing with impulse noise (like salt-and-pepper), as averaging can reduce but not eliminate such noise. <br><br>
Gaussian Filter:
The stronger presence of noise after applying the Gaussian filter is interesting. This suggests that the noise is not Gaussian in nature. Gaussian filters are optimized for reducing Gaussian noise, so their poor performance here is a strong indicator that we're dealing with a different type of noise.<br><br>
Median Filter:
The excellent performance of the median filter, removing all noises without significant image degradation, is a very strong indicator that the primary noise type in this image is salt-and-pepper noise (also known as impulse noise).<br><br>

Conclusion:
Based on these observations, we can deduce that the image most likely contains salt-and-pepper noise.<br><br>

Reasons:<br><br>
Median filters are particularly effective at removing salt-and-pepper noise while preserving edges and details. The fact that it removed all noise supports this conclusion.<br><br>
Average and Gaussian filters perform poorly on salt-and-pepper noise because they spread the effect of the noise pixels to neighboring pixels, which can actually make the noise more noticeable or create a blurry effect.<br><br>
The ineffectiveness of the Gaussian filter strongly suggests that the noise is not Gaussian in nature.

In [ ]:
# Question 3
# Load the image
image = cv.imread('images/boat.jpg')

# Convert the image to grayscale
gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

# Apply Otsu's thresholding
_, thresh = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

# Find contours in the thresholded image
contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

# Find the largest contour (assuming it's the boat with people)
largest_contour = max(contours, key=cv.contourArea)

# Get the bounding rectangle
x, y, w, h = cv.boundingRect(largest_contour)

# Draw the bounding box on the original image
result = image.copy()
cv.rectangle(result, (x, y), (x+w, y+h), (0, 255, 0), 2)

# Display the results
cv.imshow('Result with Bounding Box', result)
cv.waitKey(0)
cv.destroyAllWindows()